#### Notebook Goal: Understanding and Constructing the `AlleleProfile` Class

* The `MolecularDefinition` class serves as the parent class for `AlleleProfile`.  
* The `AlleleProfile` class inherits from the `MolecularDefinition` class.  
* The `AlleleProfile` includes specific modifications tailored for its use case, as follows:  
    * **`moleculeType` cardinality**: Changed from `0..1` to `1..1`.  
    * **`location` cardinality**: Changed from `0..*` to `1..1`.  
    * **`representation` cardinality**: Changed from `0..*` to `1..*`.  
    * **`memberState`**: This field has been removed.  
    * **`representation.focus.coding`**: Required to include `coding.code` with a default value of `allele-state` and `coding.system`.  


In [25]:
from profiles.alleleprofile import AlleleProfile

In [26]:
# Example 1 - Substitition origin: "NC_000002.12:g.27453449C>T"
example_substitition ={
    "resourceType" : "MolecularDefinition",
    "id" : "demo-example-hgvs-substitition",
    "meta" : {
      "profile" : ["http://hl7.org/fhir/StructureDefinition/allelesliced"]
    },
    "moleculeType" : {
      "coding" : [{
        "system" : "http://hl7.org/fhir/sequence-type",
        "code" : "dna",
        "display" : "DNA Sequence"
      }]
    },
    "location" : [
        {
      "sequenceLocation" : {
        "sequenceContext" : {
          "reference" : "MolecularDefinition/example-sequence-nc000002-url",
          "type" : "MolecularDefinition",
          # Example needs to contain the reference sequence
          "display" : "NC_000002.12"
        },
        "coordinateInterval" : {
          "coordinateSystem" : {
            "system" : {
              "coding" : [{
                "system" : "http://loinc.org",
                "code" : "LA30100-4",
                # Example needs to contain the systems coordinate
                "display" : "0-based interbase" 
              }],
              "text" : "0-based interbase"
            }
          },
          # Example needs to contain the startQuantity
          "startQuantity" : {
            "value" : 27453448
          },
          # Example needs to contain the endQuantity
          "endQuantity" : {
            "value" : 27453449
          }
        }
      }
    }
    ],
    "representation" : [{
      "focus" : {
        "coding" : [{
          "system" : "http://hl7.org/fhir/moleculardefinition-focus",
          "code" : "allele-state",
          "display" : "Allele State"
        }]
      },
      "literal" : {
        "value" : "T"
      }
    }]
  }

In [27]:
AlleleProfile(**example_substitition).model_dump()

{'resourceType': 'MolecularDefinition',
 'id': 'demo-example-hgvs-substitition',
 'meta': {'profile': ['http://hl7.org/fhir/StructureDefinition/allelesliced']},
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nc000002-url',
     'type': 'MolecularDefinition',
     'display': 'NC_000002.12'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interbase'}],
       'text': '0-based interbase'}},
     'startQuantity': {'value': Decimal('27453448')},
     'endQuantity': {'value': Decimal('27453449')}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir/moleculardefinition-focus',
      'code': 'allele-state',
      'display': 'Allele State'}]},
   'literal': {'value': 'T'}

##### Step 1: Defining the MolecularDefinition Location Component

In [28]:
from fhir.resources.coding import Coding

coding_val = Coding(
        system='http://loinc.org',
        code= 'LA30100-4',
        display= '0-based interbase'
    )

from fhir.resources.codeableconcept import CodeableConcept

codeconcept_val = CodeableConcept(
    coding=[coding_val],
    text = '0-based interbase'
)

from moldefresource.moleculardefinition import MolecularDefinitionLocationSequenceLocationCoordinateIntervalCoordinateSystem

MolDefLocSeqLocCoordIntCoord = MolecularDefinitionLocationSequenceLocationCoordinateIntervalCoordinateSystem(
    system=codeconcept_val
)

from moldefresource.moleculardefinition import MolecularDefinitionLocationSequenceLocationCoordinateInterval
from fhir.resources.quantity import Quantity

MolDefLocSeqLocCoordInt = MolecularDefinitionLocationSequenceLocationCoordinateInterval(
    coordinateSystem=MolDefLocSeqLocCoordIntCoord,
    startQuantity=Quantity(value=27453448),
    endQuantity=Quantity(value=27453449)
)

from moldefresource.moleculardefinition import MolecularDefinitionLocationSequenceLocation
from fhir.resources.reference import Reference

MolDefLocSeqLoc = MolecularDefinitionLocationSequenceLocation(
    sequenceContext=Reference(
        reference='MolecularDefinition/example-sequence-nc000002-url',
        type='MolecularDefinition',
        display= 'NC_000002.12'
    ),
    coordinateInterval = MolDefLocSeqLocCoordInt
)

from moldefresource.moleculardefinition import MolecularDefinitionLocation

MolDefLoc = MolecularDefinitionLocation(
    sequenceLocation = MolDefLocSeqLoc
)

MolDefLoc.model_dump()

{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nc000002-url',
   'type': 'MolecularDefinition',
   'display': 'NC_000002.12'},
  'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
       'code': 'LA30100-4',
       'display': '0-based interbase'}],
     'text': '0-based interbase'}},
   'startQuantity': {'value': Decimal('27453448')},
   'endQuantity': {'value': Decimal('27453449')}}}}

##### Step 2: Defining the MolecularDefinition Representation Component

In [29]:
from moldefresource.moleculardefinition import MolecularDefinitionRepresentationLiteral
MolDefRepLit = MolecularDefinitionRepresentationLiteral(value="T")

from fhir.resources.coding import Coding

focus_coding_val = Coding(
        system="http://hl7.org/fhir/moleculardefinition-focus",
        code= "allele-state",
        display= "Allele State"
    )

from fhir.resources.codeableconcept import CodeableConcept

focus_codeconcept_val = CodeableConcept(
    coding=[focus_coding_val],
)

from moldefresource.moleculardefinition import  MolecularDefinitionRepresentation
MolDefRep = MolecularDefinitionRepresentation(
    literal=MolDefRepLit,
    focus=focus_codeconcept_val
    )

MolDefRep.model_dump()

{'focus': {'coding': [{'system': 'http://hl7.org/fhir/moleculardefinition-focus',
    'code': 'allele-state',
    'display': 'Allele State'}]},
 'literal': {'value': 'T'}}

##### Final Step: Integrating Location, Representation and other Data into the Complete AlleleProfile Class

In [30]:
from fhir.resources.meta import Meta

meta_val = Meta(
  profile= ["http://hl7.org/fhir/StructureDefinition/allelesliced"]
)

from fhir.resources.coding import Coding

moltype_coding_val = Coding(
        system="http://hl7.org/fhir/sequence-type",
        code= "dna",
        display= "DNA Sequence"
    )

from fhir.resources.codeableconcept import CodeableConcept

moltype_codeconcept_val = CodeableConcept(
    coding=[moltype_coding_val],
)

from moldefresource.moleculardefinition import MolecularDefinition

MolDef = MolecularDefinition(
  id = 'demo-example-hgvs-substitition',
  meta = meta_val,
  moleculeType=moltype_codeconcept_val,
  location=[MolDefLoc],
  representation=[MolDefRep]
)
MolDef.model_dump()


{'resourceType': 'MolecularDefinition',
 'id': 'demo-example-hgvs-substitition',
 'meta': {'profile': ['http://hl7.org/fhir/StructureDefinition/allelesliced']},
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nc000002-url',
     'type': 'MolecularDefinition',
     'display': 'NC_000002.12'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interbase'}],
       'text': '0-based interbase'}},
     'startQuantity': {'value': Decimal('27453448')},
     'endQuantity': {'value': Decimal('27453449')}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir/moleculardefinition-focus',
      'code': 'allele-state',
      'display': 'Allele State'}]},
   'literal': {'value': 'T'}